In [43]:
from scipy import stats
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import load_model

2024-06-03 23:55:23.075805: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-03 23:55:23.109784: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-03 23:55:23.109810: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-03 23:55:23.109829: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-03 23:55:23.115628: I tensorflow/core/platform/cpu_feature_g

In [16]:
def Result_calculator(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        SBP_val = float(lines[1][12:-1])
        DBP_val = float(lines[2][12:-1])
        SBP_PR_val = float(lines[3][-4:-1])
        DBP_PR_val = float(lines[4][-4:-1])
        
        return [SBP_val, DBP_val, SBP_PR_val, DBP_PR_val]

# 각 환자별 평균 계산 - attempt 없애기
def np_to_df(res_data):
    res_by_sub = np.empty((0,5))
    sub_list = np.unique(res_data[:,0])

    for sub in sub_list:
        temp_sub_list = np.where(res_data[:,0]==sub)
        temp_res_val = res_data[temp_sub_list, 3:]
        temp_mean = np.mean(temp_res_val, axis = 1)
        temp_mean = np.insert(temp_mean, 0, sub)
        res_by_sub = np.insert(res_by_sub, res_by_sub.shape[0], temp_mean, axis = 0)
    
    res_dict = {"Subject":res_by_sub[:,0].astype(int), "SBP MAE":res_by_sub[:,1], "DBP MAE":res_by_sub[:,2], 
                   "SBP PR":res_by_sub[:,3], "DBP PR":res_by_sub[:,4]}
    res_pd = pd.DataFrame(res_dict)
    res_pd = res_pd.set_index('Subject')

    return res_pd

def res_reader(directory_path):
    # From model folders to Dataframe
    
    res_50 = np.empty((0,7), dtype = float) #axis = 3: Sub num, data_num, attempt, SBP_MAE, DBP_MAE, SBP_PR, DBP_PR
    res_100 = np.empty((0,7), dtype = float)
    res_360 = np.empty((0,7), dtype = float)
    res_720 = np.empty((0,7), dtype = float)
    res_1800 = np.empty((0,7), dtype = float)
    
    sub_res = np.empty((0,7), dtype = float)

    for root, dirs, files in os.walk(directory_path):
        for file in files:
            file_path = os.path.join(root, file)
            
            if file.endswith(".txt"):
                directory, filename = os.path.split(file_path)
                directory, attempt = os.path.split(directory)
                if int(attempt) < 2:
                    directory, data_num = os.path.split(directory)
                    directory, sub_num = os.path.split(directory)
                    result = Result_calculator(file_path)
                    temp_res = [sub_num, data_num, attempt]
                    temp_res = np.append(temp_res, result)
                    sub_res = np.insert(sub_res, sub_res.shape[0], temp_res, axis = 0)
    
    # 하나의 넘파이 배열 안에 모든 데이터 넣기 - attempt 별 데이터 모두 들어감
    for data in sub_res:
        if int(data[1]) == 50:
            res_50 = np.insert(res_50, res_50.shape[0], data, axis = 0)
        if int(data[1]) == 100:
            res_100 = np.insert(res_100, res_100.shape[0], data, axis = 0)
        if int(data[1]) == 360:
            res_360 = np.insert(res_360, res_360.shape[0], data, axis = 0)
        if int(data[1]) == 720:
            res_720 = np.insert(res_720, res_720.shape[0], data, axis = 0)
        if int(data[1]) == 1800:
            res_1800 = np.insert(res_1800, res_1800.shape[0], data, axis = 0)

    return res_50, res_100, res_360, res_720, res_1800
    
def group_res(dir_path):
    res_50_df, res_100_df, res_360_df, res_720_df, res_1800_df = res_reader(dir_path)
    
    _50 = round(np.mean(res_50_df,axis = 0),2)
    _100 = round(np.mean(res_100_df,axis = 0),2)
    _360 = round(np.mean(res_360_df,axis = 0),2)
    _720 = round(np.mean(res_720_df,axis = 0),2)
    _1800 = round(np.mean(res_1800_df,axis = 0),2)
    
    res_df = pd.concat((_50, _100, _360, _720, _1800), axis = 1).T
    res_df.index = [50, 100, 360, 720, 1800]

    return res_df

def load_dataset(data_path_list):
    x_data, y_data = np.empty((0,125)), np.empty((0,2))
    for data_dir_list in data_path_list:
        file_list = os.listdir(data_dir_list)
        for file_name in file_list:
            if file_name.endswith('.npz'):
                data = np.load(data_dir_list+"//"+file_name)
                x_data = np.concatenate((x_data, data['x'][:3240]),axis=0)
                y_data = np.concatenate((y_data, data['y'][:3240]),axis=0)
    return x_data, y_data

def find_files(folder_path, kind):
    file_root_list = []
    for (path, dir, files) in os.walk(folder_path):
        for filename in files:
            if filename.endswith(kind):
                file_root = os.path.join(path, filename)
                file_root_list.append(file_root)
    return file_root_list

### BHS

In [80]:
# BHS
dir_path = '/home/yckim/research/nibp_ppg/ex_3/model/transfer/v1_to_v2/'
sub_list = find_files(dir_path, "used_data_1.npz")
error = np.empty((0,2))

for sub in sub_list:
    a = os.path.dirname(sub)
    b = os.path.dirname(a)
    tl_data_number = int(os.path.split(b)[1])
    
    if tl_data_number == 1800:
        
        model_path = find_files(a, ".h5")
        model = load_model(model_path[0])
        data = np.load(sub)
        x_test, y_test = data['x_test'], data['y_test']

        y_pred = model.predict(x_test, verbose = 0)
        error = np.insert(error, error.shape[0], abs(y_test - y_pred), axis = 0)

less_5mmHg_pnt = round(np.count_nonzero(error[:,0]<= 5)/error.shape[0],2)
less_10mmHg_pnt = round(np.count_nonzero(error[:,0]<= 10)/error.shape[0],2)
less_15mmHg_pnt = round(np.count_nonzero(error[:,0]<= 15)/error.shape[0],2)

print(f"SBP\n<= 5mmHg : {less_5mmHg_pnt}\n<= 10mmHg : {less_10mmHg_pnt}\n<= 15mmHg : {less_15mmHg_pnt}")

less_5mmHg_pnt = round(np.count_nonzero(error[:,1]<= 5)/error.shape[0],2)
less_10mmHg_pnt = round(np.count_nonzero(error[:,1]<= 10)/error.shape[0],2)
less_15mmHg_pnt = round(np.count_nonzero(error[:,1]<= 15)/error.shape[0],2)

print(f"DBP\n<= 5mmHg : {less_5mmHg_pnt}\n<= 10mmHg : {less_10mmHg_pnt}\n<= 15mmHg : {less_15mmHg_pnt}")

SBP
<= 5mmHg : 0.73
<= 10mmHg : 0.93
<= 15mmHg : 0.98
DBP
<= 5mmHg : 0.91
<= 10mmHg : 0.99
<= 15mmHg : 1.0
